# AI 생활패턴 머신러닝 (7일치 PIR + Reed + FSR: Bed / Bath / Dining / Entrance)

이 노트북은 **7일치** 생활 로그가 들어 있는 다음 6개의 CSV 센서 데이터를 사용해서
생활 패턴(`sim_label`, 0 = 정상 / 1 = 이상상황)을 예측하는 간단한 AI 파이프라인입니다.

사용되는 파일 (Colab 기준 `/content` 위치에 업로드):

- `pir_7days_with_anomalies 4.csv`          (거실 PIR)
- `reed_bathroom_synthetic_7days 3.csv`     (화장실 문 Reed 스위치)
- `fsr_bed_synthetic_7days 7.csv`           (침대 FSR)
- `fsr_bathroom_synthetic_7days 1.csv`      (화장실 매트 FSR)
- `fsr_dining_synthetic_7days 6.csv`        (식탁 의자/매트 FSR)
- `fsr_entrance_synthetic_7days 5.csv`      (현관 매트 FSR)

⚠️ 반드시 위 이름 그대로 업로드한 뒤, 위에서부터 셀을 순서대로 실행하세요.


---

## 1. 문제 정의 및 배경

고령자 1인 가구, 맞벌이 가정의 아동, 자취생 등 **혼자 생활하는 사람**이 늘어나면서,
집 안에서 쓰러지거나, 화장실에서 장시간 움직이지 못하거나, 야간에 이상행동을 보이는 상황이
뒤늦게 발견되는 사례가 많습니다.

이 프로젝트는 **단순 센서(압력 센서 FSR 4개 + PIR + Reed 스위치)**만으로
7일간의 생활 로그를 만들고, 그 안에서 정의된 이상상황(`sim_label = 1`)을
AI가 자동으로 학습하여 탐지하는 것을 목표로 합니다.

---

## 2. 7일 시나리오 및 라벨(sim_label) 정의

각 CSV에는 `day_index`(0~6, 총 7일)와 `sim_label`(0=정상, 1=이상상황)이 포함되어 있습니다.
시나리오와 라벨 기준은 다음과 같습니다.

| day_index | 요일 (예시) | 시나리오 요약                                      | 대표 이상상황(예시)                                   | sim_label |
|----------:|------------|----------------------------------------------------|--------------------------------------------------------|-----------|
| 0         | 수요일     | 정상 생활 패턴                                     | 이상상황 없음                                         | 0         |
| 1         | 목요일     | 정상 생활 패턴                                     | 이상상황 없음                                         | 0         |
| 2         | 금요일     | 아침에 화장실에 갇힌 상황                          | 화장실 문 `Door_bathroom=1` + 매트 `FSR_bathroom` 장시간 활성 | 1         |
| 3         | 토요일     | 아이가 하교해야 하는데 집에 들어오지 않는 상황    | 하교 예상 시각 + 1시간 이후까지 현관 `FSR_entrance` 변화 없음 | 1         |
| 4         | 일요일     | 야간(새벽 3~4시)에 침대에서 움직이지 않는 상황    | 새벽 시간대 `FSR_bed` 값이 0에 가깝고 다른 센서도 무활동     | 1         |
| 5         | 월요일     | 한낮에 소파에서 너무 오래 움직이지 않는 상황      | 낮 시간대 소파/거실 `FSR_dining` + `PIR_living` 장시간 무변화 | 1         |
| 6         | 화요일     | 정상 생활 패턴                                     | 이상상황 없음                                         | 0         |

> 실제 CSV에서는 분 단위, 초 단위로 라벨이 찍혀 있으며,
> 이상상황 구간에 해당하는 시간대에는 `sim_label = 1`, 그 외에는 `0`으로 설정되어 있습니다.

이러한 **시나리오 기반 라벨링**을 바탕으로, 모델은
“어떤 시간대 + 어떤 센서 패턴이 이상상황인지”를 스스로 학습하게 됩니다.


In [1]:
# 1. 기본 라이브러리
import pandas as pd
import numpy as np
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option('display.max_columns', 50)

print("✅ 라이브러리 임포트 완료")

import matplotlib.pyplot as plt


✅ 라이브러리 임포트 완료


In [2]:
# 2. CSV 불러오기 (파일 이름이 조금 달라도, 센서 컬럼까지 확인해서 찾기)

def load_sensor_csv(pattern_list, sensor_keywords, desc="(no desc)"):
    """
    pattern_list: 파일 이름 패턴 목록 (glob)
    sensor_keywords: 이 CSV 안에 반드시 포함되어야 하는 센서 컬럼의 키워드 목록
                     예) ['FSR', 'bath'] → 컬럼 중에 'fsr'와 'bath'를 모두 포함하는 이름을 기대
    desc: 센서 설명 (로그용)
    """
    base_candidates = ["/content", "/mnt/data", "."]

    for base in base_candidates:
        for pattern in pattern_list:
            full_pattern = os.path.join(base, pattern)
            matches = sorted(glob.glob(full_pattern))
            for path in matches:
                try:
                    df_tmp = pd.read_csv(path)
                except Exception as e:
                    print(f"[load_sensor_csv] {desc}: {path} 읽기 실패 → {e}")
                    continue

                cols_lower = [c.lower() for c in df_tmp.columns]
                # sensor_keywords 중 모든 키워드를 포함하는 컬럼이 있는지 확인
                ok = False
                for c in cols_lower:
                    if all(k.lower() in c for k in sensor_keywords):
                        ok = True
                        break

                if ok:
                    print(f"loading ({desc}): {path}")
                    # timestamp는 parse_dates로 다시 읽기
                    return pd.read_csv(path, parse_dates=['timestamp'])

    # 어디에서도 찾지 못한 경우 → 현재 경로와 /content, /mnt/data의 파일 목록도 같이 출력
    msg_lines = [f"[load_sensor_csv] {desc}에 해당하는 CSV를 찾을 수 없습니다.",
                 f"  사용한 패턴: {pattern_list}",
                 f"  요구 센서 키워드: {sensor_keywords}"]
    for base in base_candidates:
        if os.path.exists(base):
            try:
                files = [f for f in os.listdir(base) if f.lower().endswith('.csv')]
                msg_lines.append(f"  {base} 내 CSV 파일: {files}")
            except PermissionError:
                pass
    raise FileNotFoundError("\n".join(msg_lines))

# PIR (거실) → PIR_living, sim_label 포함
pir = load_sensor_csv(
    pattern_list=["pir_7days_with_anomalies*.csv", "pir*7days*anomalies*.csv"],
    sensor_keywords=["pir"],  # PIR 컬럼이 들어있어야 함
    desc="PIR_living"
)

# Reed bathroom door (화장실 문 Reed 스위치)
reed_bath = load_sensor_csv(
    pattern_list=["*reed_bathroom*7days*.csv", "*reed*bathroom*.csv"],
    sensor_keywords=["door", "bath"],  # Door_bathroom 같은 컬럼 기대
    desc="Reed bathroom door"
)

# FSR bed (침대)
fsr_bed = load_sensor_csv(
    pattern_list=["fsr_bed*7days*.csv"],
    sensor_keywords=["fsr", "bed"],
    desc="FSR bed"
)

# FSR bathroom mat (화장실 매트)
fsr_bath = load_sensor_csv(
    pattern_list=["fsr_bathroom*7days*.csv", "*bathroom*synthetic*7days*.csv"],
    sensor_keywords=["fsr", "bath"],
    desc="FSR bathroom mat"
)

# FSR dining (식탁 의자/매트)
fsr_din = load_sensor_csv(
    pattern_list=["fsr_dining*7days*.csv"],
    sensor_keywords=["fsr", "din"],
    desc="FSR dining"
)

# FSR entrance (현관 매트)
fsr_ent = load_sensor_csv(
    pattern_list=["fsr_entrance*7days*.csv"],
    sensor_keywords=["fsr", "entrance"],
    desc="FSR entrance"
)

print("\nPIR:", pir.shape)
print("Reed bathroom:", reed_bath.shape)
print("FSR bed:", fsr_bed.shape)
print("FSR bathroom:", fsr_bath.shape)
print("FSR dining:", fsr_din.shape)
print("FSR entrance:", fsr_ent.shape)

pir.head()


loading: /content/pir_synthetic.csv
loading: /content/reed_bathroom_synthetic.csv
loading: /content/fsr_bed_normal.csv
loading: /content/fsr_bathroom_normal.csv
loading: /content/fsr_dining_normal.csv
loading: /content/fsr_entrance_normal.csv

PIR: (4320, 7)
Reed bathroom: (17280, 8)
FSR bed: (17280, 8)
FSR bathroom: (17280, 8)
FSR dining: (17280, 8)
FSR entrance: (17280, 8)


,timestamp,hour,minute,weekday,is_weekend,PIR_living,sim_label
0,2025-01-01 00:00:00,0,0,2,0,0,0
1,2025-01-01 00:01:00,0,1,2,0,0,0
2,2025-01-01 00:02:00,0,2,2,0,0,0
3,2025-01-01 00:03:00,0,3,2,0,0,0
4,2025-01-01 00:04:00,0,4,2,0,0,0


In [ ]:
# 2-1. 센서별 timestamp 범위 및 sync 상태 점검 + 라벨(sim_label) 분포 시각화

print("=== [Timestamp Range / Interval Check] ===")
sensor_dfs = {
    "PIR": pir,
    "Reed_bath": reed_bath,
    "FSR_bed": fsr_bed,
    "FSR_bath": fsr_bath,
    "FSR_dining": fsr_din,
    "FSR_entrance": fsr_ent,
}

for name, df_s in sensor_dfs.items():
    ts = pd.to_datetime(df_s["timestamp"])
    print(f"\n[{name}] rows={len(df_s)}")    
    print("  start:", ts.min())
    print("  end  :", ts.max())
    # 간단한 샘플링 간격 추정 (앞부분 100개 정도)
    if len(ts) > 1:
        deltas = (ts.sort_values().diff().dropna().dt.total_seconds())
        approx_step = deltas.mode().iloc[0] if not deltas.mode().empty else deltas.median()
        print(f"  approx sampling interval (sec): {approx_step}")

# PIR 기준 sim_label 분포 확인
print("\n=== [sim_label distribution in PIR data] ===")
print(pir["sim_label"].value_counts())

plt.figure()
pir["sim_label"].value_counts().sort_index().plot(kind="bar")
plt.xticks([0,1], ["정상(0)", "이상(1)"])
plt.title("sim_label 분포 (PIR 기준) - 데이터 불균형 확인용")
plt.ylabel("샘플 수")
plt.tight_layout()
plt.show()


In [3]:
# 3. 컬럼 자동 탐색 + 개별 센서 DataFrame 정리

# PIR 기준 시간 컬럼 (PIR에는 second가 없다고 가정)
BASE_TIME_COLS = ['timestamp', 'hour', 'minute', 'weekday', 'is_weekend']

def find_sensor_col(df, keywords):
    """여러 keyword 중 하나라도 포함하는 센서 컬럼 이름 찾기."""
    cols = list(df.columns)
    lower_map = {c.lower(): c for c in cols}
    for key in keywords:
        key_low = key.lower()
        for lc, orig in lower_map.items():
            if key_low in lc:
                return orig
    raise ValueError(f"센서 컬럼을 찾지 못했습니다. keywords={keywords}, columns={cols}")

# 3-1. PIR (거실) → 시간 정보 + PIR + sim_label
pir_sensor_col = find_sensor_col(pir, ['pir', 'living'])
print("🔍 PIR 센서 컬럼:", pir_sensor_col)

pir_df = pir[BASE_TIME_COLS + [pir_sensor_col, 'sim_label']].copy()
pir_df.rename(columns={pir_sensor_col: 'PIR_living'}, inplace=True)

print("PIR DF:", pir_df.shape)
pir_df.head()


🔍 PIR 센서 컬럼: PIR_living
PIR DF: (4320, 7)


,timestamp,hour,minute,weekday,is_weekend,PIR_living,sim_label
0,2025-01-01 00:00:00,0,0,2,0,0,0
1,2025-01-01 00:01:00,0,1,2,0,0,0
2,2025-01-01 00:02:00,0,2,2,0,0,0
3,2025-01-01 00:03:00,0,3,2,0,0,0
4,2025-01-01 00:04:00,0,4,2,0,0,0


In [4]:
# 4. FSR / Reed 센서 DataFrame 만들기 (timestamp + 센서값만 사용)

# 4-1. FSR Bathroom
fsr_bath_col = find_sensor_col(fsr_bath, ['bath'])
print("🔍 FSR bathroom 센서 컬럼:", fsr_bath_col)
fsr_bath_df = fsr_bath[['timestamp', fsr_bath_col]].copy()
fsr_bath_df.rename(columns={fsr_bath_col: 'FSR_bathroom'}, inplace=True)

# 4-2. FSR Bed
fsr_bed_col = find_sensor_col(fsr_bed, ['bed'])
print("🔍 FSR bed 센서 컬럼:", fsr_bed_col)
fsr_bed_df = fsr_bed[['timestamp', fsr_bed_col]].copy()
fsr_bed_df.rename(columns={fsr_bed_col: 'FSR_bed'}, inplace=True)

# 4-3. FSR Dining
fsr_din_col = find_sensor_col(fsr_din, ['din', 'dining'])
print("🔍 FSR dining 센서 컬럼:", fsr_din_col)
fsr_din_df = fsr_din[['timestamp', fsr_din_col]].copy()
fsr_din_df.rename(columns={fsr_din_col: 'FSR_dining'}, inplace=True)

# 4-4. FSR Entrance
fsr_ent_col = find_sensor_col(fsr_ent, ['entrance', 'entr'])
print("🔍 FSR entrance 센서 컬럼:", fsr_ent_col)
fsr_ent_df = fsr_ent[['timestamp', fsr_ent_col]].copy()
fsr_ent_df.rename(columns={fsr_ent_col: 'FSR_entrance'}, inplace=True)

# 4-5. Reed (Bathroom Door)
reed_bath_col = find_sensor_col(reed_bath, ['door', 'reed'])
print("🔍 Reed bathroom 센서 컬럼:", reed_bath_col)
reed_bath_df = reed_bath[['timestamp', reed_bath_col]].copy()
reed_bath_df.rename(columns={reed_bath_col: 'Door_bathroom'}, inplace=True)

print("FSR_bath_df:", fsr_bath_df.shape)
print("FSR_bed_df :", fsr_bed_df.shape)
print("FSR_din_df :", fsr_din_df.shape)
print("FSR_ent_df :", fsr_ent_df.shape)
print("Reed_bath_df:", reed_bath_df.shape)


🔍 FSR bathroom 센서 컬럼: FSR_bathroom_raw
🔍 FSR bed 센서 컬럼: FSR_bed_raw
🔍 FSR dining 센서 컬럼: FSR_dining_raw
🔍 FSR entrance 센서 컬럼: FSR_entrance_raw
🔍 Reed bathroom 센서 컬럼: Door_bathroom
FSR_bath_df: (17280, 2)
FSR_bed_df : (17280, 2)
FSR_din_df : (17280, 2)
FSR_ent_df : (17280, 2)
Reed_bath_df: (17280, 2)


In [5]:
# 5. 하나의 통합 DataFrame으로 병합

df = (
    pir_df
    .merge(fsr_bath_df, on='timestamp', how='left')
    .merge(fsr_bed_df,  on='timestamp', how='left')
    .merge(fsr_din_df,  on='timestamp', how='left')
    .merge(fsr_ent_df,  on='timestamp', how='left')
    .merge(reed_bath_df, on='timestamp', how='left')
)

print("통합 DF shape:", df.shape)
df.head()


통합 DF shape: (210240, 12)


,timestamp,hour,minute,weekday,is_weekend,PIR_living,sim_label,FSR_bathroom,FSR_bed,FSR_dining,FSR_entrance,Door_bathroom
0,2025-01-01,0,0,2,0,0,0,6.0,400.0,6.0,6.0,0.0
1,2025-01-01,0,0,2,0,0,0,6.0,400.0,19.0,6.0,0.0
2,2025-01-01,0,0,2,0,0,0,6.0,400.0,28.0,6.0,0.0
3,2025-01-01,0,0,2,0,0,0,6.0,400.0,14.0,6.0,0.0
4,2025-01-01,0,0,2,0,0,0,6.0,400.0,10.0,6.0,0.0



---

## 3. Feature Engineering 개요

이상행동 탐지는 **단순 센서 raw value**만 보는 것보다,
시간 흐름을 반영한 특성을 함께 사용하는 것이 중요합니다.

### 3.1 기본 시간/센서 feature

- `hour`, `minute`, `weekday`, `is_weekend`  
  → 생활 패턴은 시각/요일에 따라 다르므로, 같은 센서 값이라도 새벽/낮/주말 여부를 함께 고려합니다.
- `PIR_living`  
  → 거실 움직임 여부. 소파/식탁 FSR 신호와 함께 보면 **눕기/앉기/무활동** 상태를 구분하는 데 도움.
- `FSR_bathroom`, `FSR_bed`, `FSR_dining`, `FSR_entrance`, `Door_bathroom`  
  → 각각 화장실 매트, 침대, 식탁/소파, 현관 매트, 화장실 문 열림 상태를 의미합니다.

### 3.2 시간 누적 기반 feature 예시

- `bathroom_occupied_minutes`  
  → **현재 시각까지 연속해서 화장실에 머문 시간(분)**을 누적한 값입니다.
    - 평소에는 5~10분 수준에서 종료되지만,
    - 시나리오상 위험 상황(수요일 아침 갇힘)에서는 30분 이상 길어지도록 설계되어 있습니다.
    - 따라서 이 값이 급격히 커지면 **화장실 장기 체류 이상상황**으로 판단하는 중요한 단서가 됩니다.

- `PIR_edge`  
  → 거실 PIR 신호의 **변화(edge)**를 나타내는 지표입니다.
    - 0 → 1로 바뀌면 사람이 들어온/움직이기 시작한 순간,
    - 1 → 0으로 바뀌면 움직임이 끝난 순간으로 볼 수 있습니다.
    - 같은 `PIR_living` 값이라도, edge 정보를 함께 보면
      *“방금 전까지 가만히 있다가 갑자기 움직임이 시작됨”*, 
      *“오랫동안 움직이다가 갑자기 멈춤”* 같은 패턴을 구분할 수 있습니다.

이런 feature들을 통해 **센서 한 개의 순간값이 아니라,
“얼마 동안, 어떤 순서로, 어떤 조합으로” 센서가 변했는지를
학습할 수 있도록 설계**했습니다.


In [6]:
# 6. 시간 기반 feature 포함: 특징 / 라벨 분리 및 학습
#    - 화장실 체류 시간 feature 추가
#    - 거실 PIR '지나갈 때' (edge) feature 추가
#    - 날짜 단위 Train / Valid / Test 분리 (3~5일 이상 훈련, 충분한 테스트)
#    - 다양한 평가지표(Accuracy / Precision / Recall / F1) 계산

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix,
)

# 결측치(NaN)는 0으로 대체하고, 시간 순으로 정렬
df = df.fillna(0).sort_values('timestamp').reset_index(drop=True)

# 6-1. 화장실 체류 시간 feature 만들기
# FSR_bathroom 값이 어느 정도 이상이거나 문이 특정 상태일 때 '사용 중'이라고 가정
# (임계값 5.0은 예시이며, 실제 데이터에 맞게 조정 가능)
df['is_bathroom_occupied'] = ((df['FSR_bathroom'] > 5.0) | (df['Door_bathroom'] == 1)).astype(int)

# '사용 중'이 1로 연속되는 구간마다 체류 시간을 0,1,2,... 분 단위로 증가시키는 방식
group = (df['is_bathroom_occupied'] == 0).cumsum()
df['bathroom_occupied_minutes'] = df.groupby(group).cumcount()
df.loc[df['is_bathroom_occupied'] == 0, 'bathroom_occupied_minutes'] = 0

# 6-2. 거실 PIR: '지나갈 때' (0->1 변화) 를 feature로 추가
df['PIR_prev'] = df['PIR_living'].shift(1).fillna(0)
df['PIR_edge'] = ((df['PIR_prev'] == 0) & (df['PIR_living'] == 1)).astype(int)

# 6-3. feature 구성
base_feature_cols = [
    'hour', 'minute', 'weekday', 'is_weekend',
    'PIR_living',
    'FSR_bathroom', 'FSR_bed', 'FSR_dining', 'FSR_entrance',
    'Door_bathroom',
]

time_feature_cols = [
    'bathroom_occupied_minutes',  # 화장실 체류 시간(분 단위로 가정)
    'PIR_edge',                   # 거실을 '지나가는 순간'인지 여부
]

feature_cols = base_feature_cols + time_feature_cols

X = df[feature_cols].copy()
y = df['sim_label'].copy()

print("X shape:", X.shape)
print("y shape:", y.shape)
print("feature_cols:", feature_cols)

# ===== 날짜 단위로 Train / Valid / Test 분리 =====
# - timestamp의 날짜(년-월-일) 기준으로 나눔
# - 전체 날짜가 5일 이상이면:
#     * 마지막 2일: Test
#     * 그 이전 1일: Validation
#     * 나머지 처음 날짜들: Train  (=> 자연스럽게 3일 이상 훈련)
# - 날짜가 5일 미만이면: 일반적인 stratified random split로 fallback

df['date'] = df['timestamp'].dt.date
unique_dates = sorted(df['date'].unique())
n_days = len(unique_dates)
print(f"전체 날짜 수: {n_days}, 범위: {unique_dates[0]} ~ {unique_dates[-1]}")

if n_days >= 5:
    test_dates = unique_dates[-2:]          # 마지막 2일 → Test (조금 더 크게)
    val_dates  = [unique_dates[-3]]         # 그 앞의 1일 → Validation
    train_dates = unique_dates[:-3]         # 나머지 앞쪽 날짜들 → Train (>= 2일, 보통 3~5일 이상)

    train_mask = df['date'].isin(train_dates)
    val_mask   = df['date'].isin(val_dates)
    test_mask  = df['date'].isin(test_dates)

    X_train, y_train = X[train_mask], y[train_mask]
    X_val,   y_val   = X[val_mask],   y[val_mask]
    X_test,  y_test  = X[test_mask],  y[test_mask]

    print("=== 날짜 기반 분리 사용 ===")
    print("Train dates:", train_dates)
    print("Valid date :", val_dates)
    print("Test dates :", test_dates)
else:
    print("⚠️ 날짜 수가 5일 미만이어서, 일반적인 stratified random split을 사용합니다.")
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y,
        test_size=0.3,           # Test 비율을 30%로 조금 더 크게
        random_state=42,
        stratify=y
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval,
        test_size=0.2,
        random_state=42,
        stratify=y_trainval
    )

print("Train  :", X_train.shape, y_train.shape)
print("Valid  :", X_val.shape,   y_val.shape)
print("Test   :", X_test.shape,  y_test.shape)

# 6-5. 스케일링 (Train 기준으로 fit, 나머지 transform)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)


print("\n=== [Train set sim_label 분포] ===")
print(y_train.value_counts())

# 6-6. RandomForest 하이퍼파라미터 명시 & 학습
# --> 불균형 데이터에서 이상상황(sim_label=1)을 더 잘 잡기 위해 class_weight 부여
rf_params = dict(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1,
    class_weight={0: 1, 1: 10},  # 이상상황(1)에 더 큰 가중치 부여
)
print("\nRandomForest 하이퍼파라미터:", rf_params)

model = RandomForestClassifier(**rf_params)
model.fit(X_train_scaled, y_train)

# 6-7. Validation / Test 성능 측정
y_val_pred = model.predict(X_val_scaled)
y_test_pred = model.predict(X_test_scaled)

def print_scores(split_name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    prec_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')

    print(f"\n[{split_name}]")
    print(f"Accuracy        : {acc:.4f}")
    print(f"F1 (macro)      : {f1_macro:.4f}")
    print(f"F1 (weighted)   : {f1_weighted:.4f}")
    print(f"Precision (macro): {prec_macro:.4f}")
    print(f"Recall (macro)  : {recall_macro:.4f}")

print_scores("Validation", y_val, y_val_pred)
print_scores("Test",       y_test, y_test_pred)

print("\n=== [Test] Classification Report ===")
print(classification_report(y_test, y_test_pred))

print("\n=== [Test] Confusion Matrix (raw) ===")
print(confusion_matrix(y_test, y_test_pred))

# 이후 시각화/추가 분석 셀에서 사용하기 위해 y_test, y_test_pred, X_test, X_test_scaled, X, y 를 그대로 둠
y_pred = y_test_pred  # 기존 변수 이름 y_pred도 Test 기준으로 맞춰둠


X shape: (210240, 10)
y value counts:
sim_label
0    200159
1     10081
Name: count, dtype: int64
Train: (168192, 10)  Test: (42048, 10)

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     40032
           1       1.00      1.00      1.00      2016

    accuracy                           1.00     42048
   macro avg       1.00      1.00      1.00     42048
weighted avg       1.00      1.00      1.00     42048


=== Confusion Matrix ===
[[40032     0]
 [    0  2016]]


In [7]:
# 7. Feature Importance 시각화

import matplotlib.pyplot as plt
import numpy as np

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(8, 5))
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), [feature_cols[i] for i in indices], rotation=45, ha="right")
plt.ylabel("Importance")
plt.title("Feature Importance (RandomForest)")
plt.tight_layout()
plt.show()

# 표 형태로도 중요도 확인
fi_series = pd.Series(importances, index=feature_cols).sort_values(ascending=False)
fi_series


,0
hour,0.517295
PIR_living,0.215768
weekday,0.106090
FSR_bed,0.100428
FSR_bathroom,0.014619
FSR_entrance,0.014094
FSR_dining,0.013476
minute,0.012257
Door_bathroom,0.005973
is_weekend,0.000000


In [ ]:
# 8. Confusion Matrix 시각화 (Test 세트 기준)

import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
labels = model.classes_

plt.figure(figsize=(5, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(include_values=True, xticks_rotation=45, colorbar=True)
plt.title("Confusion Matrix (Test Set)")
plt.tight_layout()
plt.show()


In [ ]:
# 9. ROC Curve & Precision-Recall Curve (불균형 데이터에서 PR Curve 강조)

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

if hasattr(model, "predict_proba"):
    y_score = model.predict_proba(X_test_scaled)
    classes = model.classes_
    n_classes = len(classes)

    # ===== Binary (0/1 이상상황) vs Multi-class 처리 분리 =====
    if n_classes == 2:
        # 양성(이상상황) 클래스는 보통 1이라고 가정
        # classes 배열에서 "1"의 인덱스를 찾아 해당 확률만 사용
        try:
            pos_label = 1
            pos_idx = list(classes).index(pos_label)
        except ValueError:
            # 혹시 라벨이 [0, 1]이 아니면 마지막 클래스를 양성으로 사용
            pos_idx = 1
            pos_label = classes[pos_idx]

        y_true_bin = (y_test == pos_label).astype(int)
        y_score_pos = y_score[:, pos_idx]

        # ROC Curve
        fpr, tpr, _ = roc_curve(y_true_bin, y_score_pos)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(6, 5))
        plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.3f})")
        plt.plot([0, 1], [0, 1], linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curve (Binary)")
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(y_true_bin, y_score_pos)
        ap = average_precision_score(y_true_bin, y_score_pos)

        plt.figure(figsize=(6, 5))
        plt.plot(recall, precision, label=f"PR curve (AP = {ap:.3f})")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title("Precision-Recall Curve (Binary, class=이상상황)")
        plt.legend()
        plt.tight_layout()
        plt.show()

    else:
        # ===== Multi-class (이전 버전과 동일: micro-average) =====
        y_test_bin = label_binarize(y_test, classes=classes)

        # ROC Curve (micro-average)
        fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(6, 5))
        plt.plot(fpr, tpr, label=f"micro-average ROC (AUC = {roc_auc:.3f})")
        plt.plot([0, 1], [0, 1], linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curve (micro-average, multi-class)")
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Precision-Recall Curve (micro-average)
        precision, recall, _ = precision_recall_curve(y_test_bin.ravel(), y_score.ravel())
        ap = average_precision_score(y_test_bin, y_score, average="micro")

        plt.figure(figsize=(6, 5))
        plt.plot(recall, precision, label=f"micro-average PR (AP = {ap:.3f})")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title("Precision-Recall Curve (micro-average, multi-class)")
        plt.legend()
        plt.tight_layout()
        plt.show()
else:
    print("현재 모델은 predict_proba를 지원하지 않아 ROC/PR 곡선을 그릴 수 없습니다.")


In [ ]:
# 10. Cross-Validation (교차 검증 성능) - TimeSeriesSplit 기반

from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone

# 시간 순서(time-series)를 보존하기 위해, 랜덤 셔플 대신 TimeSeriesSplit 사용
# (과거 데이터로 학습 → 미래 구간 검증 구조)
tscv = TimeSeriesSplit(n_splits=5)

def cv_eval_model_timeseries(X, y, base_model, base_scaler, cv):
    acc_scores = []
    f1_macro_scores = []

    # X, y는 timestamp 기준으로 정렬된 df/series라고 가정
    for fold, (train_idx, test_idx) in enumerate(cv.split(X)):
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

        sc = clone(base_scaler)
        X_tr_sc = sc.fit_transform(X_tr)
        X_te_sc = sc.transform(X_te)

        m = clone(base_model)
        m.fit(X_tr_sc, y_tr)
        y_pred_cv = m.predict(X_te_sc)

        acc_scores.append(accuracy_score(y_te, y_pred_cv))
        f1_macro_scores.append(f1_score(y_te, y_pred_cv, average="macro"))

        print(f"Fold {fold}: Accuracy={acc_scores[-1]:.4f}, F1(macro)={f1_macro_scores[-1]:.4f}")

    return acc_scores, f1_macro_scores

acc_scores, f1_macro_scores = cv_eval_model_timeseries(X, y, model, scaler, tscv)

print("\n=== TimeSeriesSplit 교차 검증 결과 ===")
print("Accuracy list:", acc_scores)
print("F1(macro) list:", f1_macro_scores)
print(f"평균 Accuracy : {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"평균 F1(macro): {np.mean(f1_macro_scores):.4f} ± {np.std(f1_macro_scores):.4f}")


In [ ]:
# 11. 오류 사례 분석 (잘못 분류된 샘플 확인)

mis_idx = (y_test != y_pred)
mis_samples = X_test.loc[mis_idx].copy()
mis_samples["real_label"] = y_test[mis_idx]
mis_samples["pred_label"] = y_pred[mis_idx]

print(f"잘못 분류된 샘플 개수: {len(mis_samples)}")
# 앞에서 10개만 확인
mis_samples.head(10)


In [ ]:
# 8. 3시간 구간 / 거실 통과(PIR_edge) 기준 평가 함수

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

def evaluate_on_time_window(df, model, scaler, feature_cols, start_time, hours=3, only_pir_edge=False):
    """ 
    df           : 통합 DataFrame (timestamp, feature, sim_label 포함)
    start_time   : pd.Timestamp 또는 문자열 (예: '2024-01-01 09:00:00')
    hours        : 평가할 시간 길이 (기본 3시간)
    only_pir_edge: True면 PIR_edge == 1 (거실을 '지나갈 때')만 사용
    """
    import pandas as pd

    if isinstance(start_time, str):
        start_time = pd.to_datetime(start_time)

    end_time = start_time + pd.Timedelta(hours=hours)

    mask = (df['timestamp'] >= start_time) & (df['timestamp'] < end_time)
    df_win = df.loc[mask].copy()

    if only_pir_edge:
        df_win = df_win[df_win.get('PIR_edge', 0) == 1]

    if df_win.empty:
        print("⚠️ 해당 시간 구간에 데이터가 없습니다.")
        return

    X_win = df_win[feature_cols]
    y_win = df_win['sim_label']

    X_win_scaled = scaler.transform(X_win)
    y_pred_win = model.predict(X_win_scaled)

    print(f"⏱ 시간 범위: {start_time} ~ {end_time} (rows={len(df_win)})")
    if only_pir_edge:
        print("※ PIR_edge == 1 (거실을 '지나갈 때')인 시점만 사용")

    acc = accuracy_score(y_win, y_pred_win)
    f1_macro = f1_score(y_win, y_pred_win, average='macro')
    f1_weighted = f1_score(y_win, y_pred_win, average='weighted')

    print(f"Accuracy      : {acc:.4f}")
    print(f"F1 (macro)    : {f1_macro:.4f}")
    print(f"F1 (weighted) : {f1_weighted:.4f}")

    print("\n=== Classification Report ===")
    print(classification_report(y_win, y_pred_win))

    print("\n=== Confusion Matrix ===")
    print(confusion_matrix(y_win, y_pred_win))

print("✅ 3시간 구간 / 거실 통과 기준 평가 함수 정의 완료")

# 사용 예시:
# evaluate_on_time_window(df, model, scaler, feature_cols, start_time=df['timestamp'].min(), hours=3)
# evaluate_on_time_window(df, model, scaler, feature_cols, start_time=df['timestamp'].min(), hours=3, only_pir_edge=True)


In [ ]:
# 12. 여러 개의 Test 구간(3시간 창)을 사용한 신뢰도 평가

import numpy as np
import pandas as pd

def evaluate_multiple_windows(df, model, scaler, feature_cols,
                              window_hours=3, n_windows=5,
                              only_pir_edge=False):
    """
    데이터 전체에서 여러 개의 3시간(또는 window_hours) 구간을 뽑아서
    각 구간마다 Accuracy / F1을 계산하고, 평균과 표준편차를 출력합니다.

    - df           : 통합 DataFrame (timestamp, feature, sim_label 포함)
    - window_hours : 각 윈도우 길이 (기본 3시간)
    - n_windows    : 평가에 사용할 윈도우 개수
    - only_pir_edge: True이면 PIR_edge == 1인 행만 사용
    """
    from sklearn.metrics import accuracy_score, f1_score

    df = df.sort_values('timestamp').reset_index(drop=True)
    t_min = df['timestamp'].min()
    t_max = df['timestamp'].max()

    if pd.isna(t_min) or pd.isna(t_max):
        print("⚠️ timestamp 정보가 부족합니다.")
        return

    total_hours = (t_max - t_min).total_seconds() / 3600.0
    if total_hours < window_hours:
        print("⚠️ 전체 데이터 길이가 window_hours보다 짧습니다.")
        return

    # 시작 시각 후보들을 균등 간격으로 나눠서 뽑기
    starts = np.linspace(0, total_hours - window_hours, num=n_windows)
    acc_list = []
    f1_list = []

    for i, offset_h in enumerate(starts):
        start_time = t_min + pd.Timedelta(hours=float(offset_h))
        end_time = start_time + pd.Timedelta(hours=window_hours)

        mask = (df['timestamp'] >= start_time) & (df['timestamp'] < end_time)
        df_win = df.loc[mask].copy()

        if only_pir_edge:
            df_win = df_win[df_win.get('PIR_edge', 0) == 1]

        if df_win.empty:
            print(f"[윈도우 {i+1}] 데이터 없음: {start_time} ~ {end_time}")
            continue

        X_win = df_win[feature_cols]
        y_win = df_win['sim_label']

        X_win_scaled = scaler.transform(X_win)
        y_pred_win = model.predict(X_win_scaled)

        acc = accuracy_score(y_win, y_pred_win)
        f1m = f1_score(y_win, y_pred_win, average='macro')

        acc_list.append(acc)
        f1_list.append(f1m)

        print(f"[윈도우 {i+1}] {start_time} ~ {end_time}")
        print(f"  Accuracy : {acc:.4f}, F1(macro): {f1m:.4f}")

    if len(acc_list) == 0:
        print("⚠️ 유효한 윈도우가 없어 평균을 계산할 수 없습니다.")
        return

    print("\n=== 여러 Test 윈도우 요약 ===")
    print(f"평균 Accuracy : {np.mean(acc_list):.4f} ± {np.std(acc_list):.4f}")
    print(f"평균 F1(macro): {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

print("✅ 여러 개의 3시간 Test 윈도우 평가 함수 정의 완료")

# 8. 행동 예측 함수 및 모델/스케일러 저장

아래 코드 셀에서는:

1. 학습된 `model`, `scaler`, `feature_cols`를 파일로 저장하고 (`activity_model.pkl`, `scaler.pkl`, `feature_cols.pkl`)
2. 테스트 데이터에 대한 예측 결과 일부를 출력하는 `show_example_predictions` 함수를 정의하고
3. 새로운 센서 입력(딕셔너리 형태)을 넣으면 행동 라벨을 예측하는 `predict_activity_from_sensor` 함수를 정의합니다.

> ⚠️ 먼저 **위의 1~7번 셀을 모두 실행한 뒤** 이 셀들을 실행해야 합니다.


---

## 11. 실시간 시스템 구성도 및 Deployment 흐름

이 노트북에서 학습한 RandomForest 모델은,
실제 IoT 환경에서 다음과 같은 흐름으로 사용될 수 있습니다.

1. **센서 계층 (Edge, 집 안)**
   - ESP32 + FSR 4개 (침대 / 화장실 / 식탁·소파 / 현관)
   - ESP32 + PIR (거실)
   - ESP32 + Reed 스위치 (화장실 문, 현관 문 등)
   - 약 5초 간격으로 센서 값을 읽어서 WiFi로 전송

2. **게이트웨이 / 클라우드 계층**
   - Blynk / MQTT / HTTP 등의 프로토콜로 센서 값을 서버(또는 Colab API)로 전송
   - 서버에서는 센서 raw 값을 받으면
     - timestamp 기준으로 정렬
     - 이 노트북에서 정의한 것과 동일한 방식으로 feature를 생성
       (`bathroom_occupied_minutes`, `PIR_edge` 등)

3. **AI Inference 계층**
   - 이 노트북에서 저장한 `activity_model.pkl`, `scaler.pkl`, `feature_cols.pkl`을 로드
   - 새로 들어온 feature 벡터에 대해
     - `scaler.transform()`으로 스케일링 후
     - `model.predict()`와 `model.predict_proba()` 실행
   - `sim_label=1`로 예측되거나, 위험 확률이 임계값(예: 0.7) 이상이면
     → 이상상황으로 판정

4. **알림/대응 계층**
   - 보호자/가족 스마트폰 앱으로 푸시 알림 전송
   - 콜센터/케어 매니저에게도 이메일/대시보드 알림
   - 필요 시, 119/응급 연락망과 연계 가능

이와 같은 구조를 통해
**저비용 센서 + 경량 AI 모델**만으로도
1인 가구·아동·노인의 안전을 실시간으로 모니터링할 수 있습니다.


In [ ]:
import joblib

def save_model_and_scaler(model, scaler, feature_cols,
                          model_path="activity_model.pkl",
                          scaler_path="scaler.pkl",
                          feature_cols_path="feature_cols.pkl"):
    """학습된 모델, 스케일러, feature 리스트를 파일로 저장합니다."""
    joblib.dump(model, model_path)
    joblib.dump(scaler, scaler_path)
    joblib.dump(feature_cols, feature_cols_path)
    print(f"✅ 모델 저장: {model_path}")
    print(f"✅ 스케일러 저장: {scaler_path}")
    print(f"✅ feature 목록 저장: {feature_cols_path}")


def show_example_predictions(model, scaler, X_test, y_test, n=10):
    """테스트 데이터 앞에서부터 n개에 대한 실제 라벨 vs 예측 라벨을 출력합니다."""
    n = min(n, len(X_test))
    sample = X_test.iloc[:n].copy()
    sample_scaled = scaler.transform(sample)

    preds = model.predict(sample_scaled)

    print("=== 예측 결과 예시 (앞에서부터 {0}개) ===".format(n))
    for i in range(n):
        real_label = y_test.iloc[i]
        pred_label = preds[i]
        print(f"{i:2d}번째 샘플 -> 실제: {real_label} / 예측: {pred_label}")


def predict_activity_from_sensor(model, scaler, feature_cols, sensor_dict):
    """    센서 입력(딕셔너리)을 받아서 행동 라벨을 예측합니다.

    Parameters
    ----------
    model : 학습된 분류 모델 (예: RandomForestClassifier)
    scaler : 학습 때 사용한 StandardScaler
    feature_cols : 학습에 사용한 feature 이름 리스트
    sensor_dict : {특성 이름: 값} 형태의 딕셔너리
                  예) {
                        "hour": 13,
                        "minute": 25,
                        "weekday": 2,
                        "is_weekend": 0,
                        "PIR_living": 1,
                        "FSR_bathroom": 0.0,
                        "FSR_bed": 120.5,
                        "FSR_dining": 0.0,
                        "FSR_entrance": 10.0,
                        "Door_bathroom": 0,
                      }
    """
    # feature_cols 순서대로 값을 꺼내서 2차원 리스트로 만듭니다.
    # feature_cols에 없는 키는 0.0으로 채워 넣어서 KeyError를 방지합니다.
    row = [[sensor_dict.get(col, 0.0) for col in feature_cols]]
    X_new = pd.DataFrame(row, columns=feature_cols)

    # 학습 때와 동일하게 스케일링
    X_new_scaled = scaler.transform(X_new)

    # 예측
    pred_label = model.predict(X_new_scaled)[0]

    # (선택) 각 클래스별 확률도 반환
    proba = None
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X_new_scaled)[0]

    return pred_label, proba


# 예시: 위에서 학습된 model, scaler, feature_cols, X_test, y_test가 있다고 가정하고 사용
print("✅ 예측/저장용 함수 정의 완료")

# (선택) 모델/스케일러/feature 저장
# save_model_and_scaler(model, scaler, feature_cols)


In [ ]:
# 9. 예시: 테스트 데이터 일부를 이용한 예측 결과 확인 & 센서 입력 예측

# 1) 테스트 데이터 앞부분 n개에 대한 실제 vs 예측
show_example_predictions(model, scaler, X_test, y_test, n=10)

# 2) 임의의 센서 입력 예시 (실제 서비스에서는 여기 값들을 센서에서 받아오면 됨)
example_sensor = {
    "hour": 21,
    "minute": 30,
    "weekday": 4,      # 0=월요일 기준
    "is_weekend": 0,
    "PIR_living": 1,
    "FSR_bathroom": 0.0,
    "FSR_bed": 200.0,
    "FSR_dining": 0.0,
    "FSR_entrance": 5.0,
    "Door_bathroom": 0,
}

pred_label, proba = predict_activity_from_sensor(model, scaler, feature_cols, example_sensor)

print("\n=== 새 센서 입력에 대한 예측 ===")
print(f"예측된 행동 라벨: {pred_label}")

if proba is not None:
    print("\n각 클래스별 확률:")
    for cls_name, p in zip(model.classes_, proba):
        print(f"  {cls_name}: {p:.3f}")


In [ ]:

import joblib
joblib.dump(model, "activity_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(feature_cols, "feature_cols.pkl")
print("PKL saved!")
